# CityBikes

Need to add the relevant libraries first


In [1]:
import requests
import os
import pandas as pd

Explore the structure of the API, query the API and understand the data returned.


In [2]:
# API End points
# http://api.citybik.es/v2/networks
# {
#   "networks": [
#     {
#         "company": "JCDecaux", 
#         "href": "/v2/networks/velib", <--- network API endpoint
#         "location": {
#           "latitude": 48.856612, 
#           "city": "Paris", 
#           "longitude": 2.352233, 
#           "country": "FRA"
#         }, 
#         "name": "Vélib'", 
#         "id": "velib"
#     },
#     {...}
#   ]
# }
# http://api.citybik.es/v2/networks/network_id

# {
#   "network": {
#     "name": "Vélib'", 
#     "stations": [
#       {
#           "name": "00903 - QUAI MAURIAC  / PONT DE BERCY",   /   UTC Zulu timestamp of the last time
#           "timestamp": "2014-04-14T12:10:17.622Z",  <-------/ the station was updated on our systems
#           "longitude": 2.374340554605615,                       
#           "free_bikes": 1,   <-------------------------- Available bikes
#           "latitude": 48.83713368945151, 
#           "empty_slots": 19,  <------------------------- Empty spaces
#           "id": "f5a551a87eec15155d6409fe9d0ff8e2" <---- Unique id for this station
#       },
#       {...}
#     ], 
#     "company": "JCDecaux",           |
#     "href": "/v2/networks/velib",       |        Redundant Information
#     "location": {                    |
#       "latitude": 48.856612,         |    Just so you know where you are
#       "city": "Paris",               |---      with less requests
#       "longitude": 2.352233,         | 
#       "country": "FRA"               |    This can be filtered by passing
#     },                               |      ?fields=stations to the URI
#     "id": "velib"                    |
#   }
# }


Send a request to CityBikes for the city of your choice. 


In [3]:
# Step 1: Choose a city and get the network ID from the /v2/networks endpoint
city_name = 'Vancouver'
networks_url = 'http://api.citybik.es/v2/networks'
response = requests.get(networks_url)
networks_data = response.json()

In [4]:
# Find the network information for the chosen city
network_info = next((network for network in networks_data['networks'] if network['location']['city'] == city_name), None)

if network_info:
    # Step 2: Use the network ID to retrieve information about bike stations in that city
    network_id = network_info['id']
    api_url = f'http://api.citybik.es/v2/networks/{network_id}'

    response = requests.get(api_url)
    data = response.json()

In [5]:
#exploring how response work
print(type(data))
print(type(data['network']['stations']))
data['network']['stations'][0].keys()

<class 'dict'>
<class 'list'>


dict_keys(['empty_slots', 'extra', 'free_bikes', 'id', 'latitude', 'longitude', 'name', 'timestamp'])

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [6]:
data['network']['stations'][10]

{'empty_slots': 17,
 'extra': {'ebikes': 0,
  'has_ebikes': True,
  'last_updated': 1702153470,
  'normal_bikes': 9,
  'renting': True,
  'returning': True,
  'slots': 26,
  'uid': '0016'},
 'free_bikes': 9,
 'id': 'bf8408067b0e0c963f3ff526977bcef3',
 'latitude': 49.287097,
 'longitude': -123.118551,
 'name': 'Pender & Burrard',
 'timestamp': '2023-12-09T20:30:27.019000Z'}

In [7]:
# Step 3: Extract relevant station information
station_data = []
for station in data['network']['stations']:
        station_info = {
            'latitude': station['latitude'],
            'longitude': station['longitude'],
            'num_bikes': station['free_bikes'],
            
        }
        station_data.append(station_info)

Put your parsed results into a DataFrame.

In [8]:
    # Step 4: Create a Pandas DataFrame
    df = pd.DataFrame(station_data)


In [9]:
 # Step 5: Explore and Analyze the Data
    # Display the first few rows of the DataFrame
print(df.head())

    # Basic statistics
print(df.describe())


    latitude   longitude  num_bikes
0  49.262487 -123.114397         22
1  49.274566 -123.121817          5
2  49.279764 -123.110154         25
3  49.260599 -123.113504          2
4  49.264215 -123.117772          7
         latitude   longitude   num_bikes
count  248.000000  248.000000  248.000000
mean    49.271780 -123.119346    9.040323
std      0.011708    0.032818    5.782704
min     49.238105 -123.252261    0.000000
25%     49.263484 -123.138182    4.000000
50%     49.271063 -123.118073    8.000000
75%     49.280662 -123.099747   13.000000
max     49.302577 -123.056909   31.000000


In [10]:
response_data = data['network']['stations']
bike_stations = pd.json_normalize(response_data)

# drop unnecessary columns
columns_to_drop = list(bike_stations.columns[6:])
bike_stations = bike_stations.drop(columns=columns_to_drop)

In [11]:
bike_stations.head()

,empty_slots,free_bikes,id,latitude,longitude,name
0,13,22,7a19c49f486d7c0c02b3685d7b240448,49.262487,-123.114397,10th & Cambie
1,11,5,32603a87cfca71d0f7dfa3513bad69d5,49.274566,-123.121817,Yaletown-Roundhouse Station
2,1,25,6d42fa40360f9a6b2bf641c7b8bb2862,49.279764,-123.110154,Dunsmuir & Beatty
3,13,2,66f873d641d448bd1572ab086665a458,49.260599,-123.113504,12th & Yukon (City Hall)
4,9,7,485d4d24c803cfde829ab89699fed833,49.264215,-123.117772,8th & Ash


In [12]:
bike_stations.shape

(248, 6)

In [13]:
bike_stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   empty_slots  248 non-null    int64  
 1   free_bikes   248 non-null    int64  
 2   id           248 non-null    object 
 3   latitude     248 non-null    float64
 4   longitude    248 non-null    float64
 5   name         248 non-null    object 
dtypes: float64(2), int64(2), object(2)
memory usage: 11.8+ KB


In [14]:
#save data to use in another notebook 
bike_stations.to_csv('../data/bike_stations_van.csv')